## Multiparty XGBoost with Centralized Training
In this exercise, we'll demonstrate a workflow in which each party has its own data and sends a copy of its data to the central server. Therefore, all the training data is sent over the network to the central server, who collects it and locally trains a model on all the data. The central server will then broadcast the trained model back to the parties, who will load the model and test it on their local test datasets. 

![title](img/exercise2.png)


We will also measure the number of bytes sent over the network to show the large bandwidth needed for this workflow. 
This shows the benefits of using as much data as possible to make the model more robust.

### Data Transfer
Import the necessary libraries

In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from Utils import scp

Though we don't need to do this part, we think it's helpful to see how many bytes would be transferred over the network if you weren't the aggregator and had to send your training data over the network. Send the training data you used in Exercise 1 over the network to your `~/shared_data` directory. Note how many bytes are transferred.

* Training data for the hospital dataset is at `/data/hospital/hospital_training_{party_id}.csv`
* Training data for the insurance dataset is at `/data/insurance/insurance_training_{party_id}.csv`

In [ ]:
# Make sure you use the training data you used in exercise 1
training_data = "/path/to/training_data" # TODO: fill in the path to the training data
my_ip = "aggregator_ip" # TODO: fill in the IP of the aggregator
dest_dir = "~/shared_data"
scp(training_data, my_ip, dest_dir)

### Aggregate the Received Data
Wait for all parties to send your their data and load all the data that has been sent to your machine. For example, if three other parties sent you data, make 4 calls to `read_csv()`: one for your own data and three for the other parties' data.

Concatenate all the data in preparation for training

In [ ]:
# TODO: add training data to the argument passed to pd.concat()
training_data_lst = [aggregator_training_data, p2_training_data, ]

aggregated_training_data = pd.concat(training_data_lst) 
aggregated_training_data.shape

In [ ]:
# TODO: Split the aggregated training data into features and labels
y_agg_train = aggregated_training_data.iloc[:, 0]
x_agg_train = aggregated_training_data.iloc[:, 1:]

### Train a Model

In [ ]:
arg1, arg2 = x_agg_train, y_agg_train # TODO: fill these variables in with the aggregated features and labels

multiparty_model = xgb.XGBClassifier()
multiparty_model.fit(arg1, arg2)

### Broadcast the Trained Model
Save the trained model and send it to all parties in the federation.

In [ ]:
multiparty_model.save_model("ex2_model.model")

In [ ]:
# If you're the central server, run this cell as many times as needed to send the saved model
# to all parties in the federation
model_file = "ex2_model.model"
dest_dir = "~"

# TODO: fill in the IP addresses of all members of your federation
dest_ips = []

for ip in dest_ips:
    scp(model_file, ip, dest_dir)

### Model Evaluation

In [ ]:
# TODO: load in your local test data and preprocess it to split it into features and labels
test_data_subset = pd.read_csv('/data/hospital/hospital_test_1.csv', sep=",", header=None)
y_test = test_data_subset.iloc[:, 0]
x_test = test_data_subset.iloc[:, 1:]

In [ ]:
arg1, arg2 = x_test, y_test # TODO: set arg1 to the test features, arg2 to the test labels
preds = multiparty_model.predict(arg1)
print(accuracy_score(arg2, preds))

Discuss the results with other members of your federation. How did the centrally trained model perform on your local test data compared with the locally trained model? Did adding more data help?

Once you're ready, please move to [Exercise 3](Exercise 3.ipynb).